This notebook will be used to prototype various ideas to be used in the Kaggle IEEE CIS Fraud-Detection competition. It is unlikely to be very well organised or well annotated as I will play around with ideas as I get sparks of inspiration.

# Leaderboard score proxy

The first thing we want to do is to have an internal validation protocol that serves as a good proxy for the leaderboard score. There's no point in training models and tuning them if the metric we use isn't consistent with the dyanmics of the leaderboard score.

## Simple Train-Test split

This is the first idea we will be investigating. We will use 80% of the data for training and cross-validating to choose hyperparameters and the remaining 20% to test our model fit. We wil use xgboost as our prototyping model.

sklearn has the train_test_split function to help us. An important question to ask is whether or not we should be shuffling the data. This is strictly speaking time-series, so shuffling data may not be appropriate, but this is transactional data so any time dependencies are unlikely to be very strong.

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
train = pd.read_csv('Data/train_transaction.csv')

We'll also need to do some data pre-processing as well. Nothing fancy, transformation of categorical variables into dummies and random forward filling of NAs.

In [3]:
#Creating function to deal with NAs by shuffling and forward filling.

def ffill(df):
    
    t0 = time.time()
    
    na_count = df.isna().sum().sum()
    while na_count>0:
        df = df.sample(frac=1)
        df = df.fillna(method='ffill',limit=10)
        na_count = df.isna().sum().sum()

        df = df.sort_index()

    t1 = time.time()

    print(t1-t0)

In [4]:
#Only using the first 10000 rows otherwise my 11 inch 2014 MacBook Air won't be able to handle it :/
train_sub = train.iloc[:12000,:]
fraud = train_sub['isFraud']
train_sub = train_sub.drop('isFraud', axis=1)

#Numerics
numerics = train_sub.select_dtypes(exclude='object')
numerics = ffill(numerics)

#Converting 
categorical = train_sub.select_dtypes(include='object')
dummies = pd.get_dummies(categorical)

X = pd.concat([numerics, dummies], axis=1)

0.6911499500274658


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X.iloc[:10000,:],fraud.iloc[:10000],test_size=0.2)

In [6]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [7]:
#Setting up model evaluation
model = xgb.XGBClassifier(
    max_depth = 300,
    learning_rate = 0.1,
    objective = 'binary:logistic',    
)

model.fit(X_train, y_train, eval_metric = 'auc')

train_preds = model.predict_proba(X_train)
test_preds = model.predict_proba(X_test)

train_score = roc_auc_score(y_train, train_preds[:,1])
test_score = roc_auc_score(y_test, test_preds[:,1])

print(train_score,test_score)

0.954813162000562 0.7984707968718203


Ok, so using a train test split is not too bad, though as expected, the training score is slightly lower than the test score. sklearn's train_test_split shuffles the data for us, I'm not sure if that is the right way to go, I'll check with an unshuffled 'final' test set.

In [8]:
final_test_preds = model.predict_proba(X.iloc[10000:12000,:])
final_test_score = roc_auc_score(fraud.iloc[10000:12000], final_test_preds[:,1])
print(train_score, test_score, final_test_score)

0.9527468290321242 0.8084626584750878 0.7565747861113885


Well, that's certainly interesting. The test score on the unshuffled holdout set is even worse! This may mean that shuffling our data biases our in sample metrics upwards. I'm going to do the same thing now, but with an unshuffled train, test split.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X.iloc[:10000,:],fraud.iloc[:10000],test_size=0.2, shuffle=False)

model.fit(X_train, y_train, eval_metric = 'auc')

train_preds = model.predict_proba(X_train)
test_preds = model.predict_proba(X_test)
final_test_preds = model.predict_proba(X.iloc[10000:12000,:])

train_score = roc_auc_score(y_train, train_preds[:,1])
test_score = roc_auc_score(y_test, test_preds[:,1])
final_test_score = roc_auc_score(fraud.iloc[10000:12000], final_test_preds[:,1])

print(train_score, test_score, final_test_score)

0.9522543059777102 0.8063564101005598 0.7520473384600019


Shuffling doesn't seem to have any effect on the variation in the scores across the train, test and final test set. Rather it is the distance in time between the training set and the test set which causes the loss in generality. Let's see if we can test this. 

In [14]:
train_sub = train.iloc[:500000,:]
fraud = train_sub['isFraud']
train_sub = train_sub.drop('isFraud', axis=1)

#Numerics
numerics = train_sub.select_dtypes(exclude='object')
numerics = ffill(numerics)

#Converting 
categorical = train_sub.select_dtypes(include='object')
dummies = pd.get_dummies(categorical)

X = pd.concat([numerics, dummies], axis=1)

85.33768796920776


In [15]:
X_train = X.iloc[:10000,:]
y_train = fraud.iloc[:10000]

model.fit(X_train, y_train, eval_metric = 'auc')

scores = []
for index in np.arange(1,51):
    start = (index - 1)*10000
    end = index*10000
    preds = model.predict_proba(X.iloc[start:end,:])
    test = fraud.iloc[start:end]
    score = roc_auc_score(test,preds[:,1])
    scores.append(score)

In [16]:
print(scores, np.mean(scores))

[0.950081499355564, 0.7298494235008715, 0.7748407579423879, 0.7229818550227896, 0.7462827915059855, 0.6405861798087602, 0.6623785401786042, 0.6961530062296728, 0.7087082396486528, 0.7146374394408705, 0.6942619087999866, 0.7116187101284683, 0.7059114336772252, 0.6960016317669808, 0.7484376383523085, 0.6980731145271961, 0.7118091421380894, 0.6944058671973327, 0.7084461513526232, 0.6587806185728995, 0.6510216512686985, 0.7156446348254067, 0.6896232510442741, 0.7653221341091455, 0.6799741354535875, 0.7416376251654897, 0.7044310795433357, 0.6828547674233509, 0.7006853347996878, 0.6885068996306843, 0.6890444836654885, 0.6954904720436965, 0.7086646775656747, 0.6580491053241829, 0.5893231725089038, 0.6254057830322667, 0.6930426184484557, 0.7246621265141835, 0.6780857280076844, 0.6940381033276251, 0.684339899997854, 0.6889922991469695, 0.7990470051314383, 0.6966977888160594, 0.7270031330618443, 0.7090928379865444, 0.6799869548622548, 0.701391278796333, 0.6776327854572131, 0.6779387384958977] 0.

Seems to be the case here. While for the next 10000 observations it seems to do quite well, beyond that the performance seems to deteriorate. Even with 10000 data points evaluated in each block, the auc score fluctuates quite a bit. 

Ideally we would want an internal test auc score that corelates well with the Kaggle auc score, but I'm not sure if there's a way to guarantee that without using an absurd amount of data. We might just have to be content with it as is. 

## k-fold cross-validation

The next validation protocol we'll consider using is k-fold cross-validation. We want to see how good a proxy the cross-validation score is for the true test score. For now, I'll conduct 3-fold cross-validation on the first 10000 observations and take the average of the validation scores as our proxy. Then we'll compare it to the auc scores on data blocks of the same size to see if there are any major discrepancies.

In [10]:
from sklearn.model_selection import cross_val_predict

preds = cross_val_predict(model, X_train, y_train, method='predict_proba', fit_params={'eval_metric':'auc'})

print(roc_auc_score(y_train,preds[:,1]))

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.8017874620848717


Biased upwards, to be expected. The average scores on the test sets is closer to 0.7, quite a significant difference. This illustrates the danger of tuning to the cross-val score. Any such tweaks may run the risk of tuning to the noise in a particular data block, and probably won't generalise well.